In [2]:
# Code Cell 1: Install Required Tools
print("Step 1: Installing BWA, SAMtools, and Bowtie2.")
!apt-get install -qq bwa samtools > /dev/null
!apt-get install -qq bowtie2 > /dev/null
print("   -> Installation complete.")

Step 1: Installing BWA, SAMtools, and Bowtie2.
   -> Installation complete.


In [3]:
# Code Cell 2: Create and Navigate to Working Directory
!mkdir -p /content/reference_chrY_analysis
%cd /content/reference_chrY_analysis
print("\nStep 2: Changed directory to /content/reference_chrY_analysis")

/content/reference_chrY_analysis

Step 2: Changed directory to /content/reference_chrY_analysis


In [4]:
# Code Cell 3: Download and Decompress hg38 Chromosome Y Reference
print("\nStep 3: Downloading and unzipping hg38 Chromosome Y (chrY.fa.gz)...")
!wget -q https://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chrY.fa.gz
!gunzip -f chrY.fa.gz
print("   -> Download and decompression complete. File: chrY.fa")


Step 3: Downloading and unzipping hg38 Chromosome Y (chrY.fa.gz)...
   -> Download and decompression complete. File: chrY.fa


In [5]:
# Code Cell 4: Samtools Index and Get Length
# Samtools indexes the full FASTA file (chrY.fa) to create chrY.fa.fai
!samtools faidx chrY.fa

# Extract the length of chrY
!CHR_Y_LEN=$(awk '{if ($1=="chrY") print $2}' chrY.fa.fai)
!echo "Chromosome Y Length: $CHR_Y_LEN"

# Save the length to a file for Python to read
!echo $CHR_Y_LEN > chrY_length.txt

print("Step 4: Full chrY indexed and length extracted.")

Chromosome Y Length: 
Step 4: Full chrY indexed and length extracted.


# # Step 5: Indexing Both Aligners on the Full Chromosome Y

In [6]:
# Code Cell 6: BWA and Bowtie2 Index the FULL Reference Genome
# Indexing the full chromosome Y for both aligners.
!bwa index chrY.fa
!bowtie2-build chrY.fa chrY_index

print("\nStep 6: BWA and Bowtie2 Indexing of FULL chrY Complete.")

[bwa_index] Pack FASTA... 0.66 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=114454830, availableWord=20053256
[BWTIncConstructFromPacked] 10 iterations done. 33078222 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 61107982 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 86016862 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 108151902 characters processed.
[bwt_gen] Finished constructing BWT in 44 iterations.
[bwa_index] 166.20 seconds elapse.
[bwa_index] Update BWT... 0.35 sec
[bwa_index] Pack forward-only FASTA... 0.88 sec
[bwa_index] Construct SA from BWT and Occ... 17.92 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index chrY.fa
[main] Real time: 187.558 sec; CPU: 186.014 sec
Settings:
  Output files: "chrY_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket si

In [7]:
# Code Cell 7: Generate 45 bp CLEAN Sequence + Insert 5 'N's
import random
import os
import subprocess

CLEAN_LENGTH = 45
TOTAL_LENGTH = 50
REGION_START = 22000000
REGION_END = 23000000
found_clean_sequence = False

print("\nStep 7: Searching for a clean sequence and inserting 5 'N's...")

while not found_clean_sequence:
    # Choose a random 45 bp stretch within the constrained, reliable region
    random_start = random.randint(REGION_START, REGION_END - CLEAN_LENGTH)
    coord = f"chrY:{random_start}-{random_start + CLEAN_LENGTH - 1}"

    # Extract the sequence
    try:
        sequence_output = subprocess.check_output(
            f"samtools faidx chrY.fa {coord}", shell=True, text=True
        ).splitlines()[1]

        # Check if the sequence is clean
        if 'N' not in sequence_output and 'n' not in sequence_output:
            found_clean_sequence = True
            clean_seq = list(sequence_output)

            # Insert 5 'N' characters randomly into the 50 positions
            n_indices = random.sample(range(TOTAL_LENGTH), 5)

            final_seq = []
            clean_idx = 0
            for i in range(TOTAL_LENGTH):
                if i in n_indices:
                    final_seq.append('N')
                else:
                    final_seq.append(clean_seq[clean_idx])
                    clean_idx += 1

            final_seq_str = "".join(final_seq)

            # Format and save the modified sequence
            final_query = f""">chrY_5N_random_50bp
{final_seq_str}
"""
            with open("query_chrY_5N.fa", "w") as f:
                f.write(final_query)

            # Save the original start position (for the 45 bp) for verification
            with open("random_pos.txt", 'w') as f:
                f.write(str(random_start))

            print(f"   -> Clean sequence found at {random_start}. 5 'N's inserted.")

    except Exception as e:
        pass

print("Step 7: 50 bp sequence with 5 'N's prepared.")


Step 7: Searching for a clean sequence and inserting 5 'N's...
   -> Clean sequence found at 22095384. 5 'N's inserted.
Step 7: 50 bp sequence with 5 'N's prepared.


# # Step 8: Perform BWA-MEM Alignment

In [8]:
# Code Cell 9: Perform BWA Alignment
!bwa mem chrY.fa query_chrY_5N.fa > alignment_bwa_chrY_random.sam

print("\nStep 9: BWA-MEM Alignment complete.")

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 1 sequences (50 bp)...
[M::mem_process_seqs] Processed 1 reads in 0.000 CPU sec, 0.000 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem chrY.fa query_chrY_5N.fa
[main] Real time: 0.104 sec; CPU: 0.094 sec

Step 9: BWA-MEM Alignment complete.


In [9]:
# Code Cell 10: View and Interpret BWA Alignment Results
print("\nBWA Alignment Results (SAM format):")
!head alignment_bwa_chrY_random.sam


BWA Alignment Results (SAM format):
@SQ	SN:chrY	LN:57227415
@PG	ID:bwa	PN:bwa	VN:0.7.17-r1188	CL:bwa mem chrY.fa query_chrY_5N.fa
chrY_5N_random_50bp	4	*	0	0	*	*	0	0	AAGTTNTCNANAAAGACTNCATGCCTACAATGTCTAGGCTGGCCTGNAGA	*	AS:i:0	XS:i:0


# # Step 11: Perform Bowtie2 Alignment

In [10]:
# Code Cell 12: Perform Bowtie2 Alignment
!bowtie2 -x chrY_index -f query_chrY_5N.fa -S alignment_bowtie2_chrY_random.sam

print("\nStep 12: Bowtie2 Alignment complete.")

1 reads; of these:
  1 (100.00%) were unpaired; of these:
    1 (100.00%) aligned 0 times
    0 (0.00%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
0.00% overall alignment rate

Step 12: Bowtie2 Alignment complete.


In [11]:
# Code Cell 13: View and Interpret Bowtie2 Alignment Results
print("\nBowtie2 Alignment Results (SAM format):")
!head alignment_bowtie2_chrY_random.sam


Bowtie2 Alignment Results (SAM format):
@HD	VN:1.0	SO:unsorted
@SQ	SN:chrY	LN:57227415
@PG	ID:bowtie2	PN:bowtie2	VN:2.4.4	CL:"/usr/bin/bowtie2-align-s --wrapper basic-0 -x chrY_index -f query_chrY_5N.fa -S alignment_bowtie2_chrY_random.sam"
chrY_5N_random_50bp	4	*	0	0	*	*	0	0	AAGTTNTCNANAAAGACTNCATGCCTACAATGTCTAGGCTGGCCTGNAGA	IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII	YT:Z:UU


In [12]:
# Code Cell 14: Layman's Interpretation of Alignment Results

import os

def interpret_alignment(sam_file, tool_name):
    # Helper to parse the alignment line
    with open(sam_file, 'r') as f:
        alignment_line = next((line for line in f if not line.startswith('@')), None)

    if not alignment_line:
        return [f"ERROR: No alignment data found in {sam_file}."]

    fields = alignment_line.split('\t')

    # Extract key SAM fields
    rname = fields[2]
    pos = fields[3]
    mapq = fields[4]
    cigar = fields[5]
    flag = int(fields[1])

    # Determine status based on SAM flag
    if flag & 4:
        status = "The 50-base pair sequence COULD NOT be reliably mapped (found) on Chromosome Y."
        start_position = "N/A"
        quality_score = "N/A"
        match_details = "N/A"
    else:
        status = "The 50-base pair sequence WAS SUCCESSFULLY mapped (found) on Chromosome Y."
        start_position = f"It starts at position **{pos}** on the chromosome."
        quality_score = f"The confidence score (MAPQ) is **{mapq}** out of 60. A high score means the tool is very sure this is the right spot."

        # Analyze CIGAR string
        match_details = f"The alignment description (CIGAR) is **{cigar}**."
        match_details += "\n  - 'M' (Match) indicates how many bases of the sequence line up with the chromosome."
        match_details += "\n  - The 5 'N's were handled, resulting in this specific CIGAR and alignment score."

    # Get expected position
    try:
        with open('random_pos.txt', 'r') as f:
            expected_pos = f.read().strip()
    except:
        expected_pos = "N/A (File missing)"


    return [
        f"--- **{tool_name} Alignment Summary (Layman's Terms)** ---",
        f"1. **The Question:** Can our 50-base sequence (with 5 unknown parts) be found on Chromosome Y?",
        f"2. **The Answer:** {status}",
        f"3. **Start Location:** {start_position}",
        f"4. **Original Start:** The sequence was taken starting at **{expected_pos}** (the map position should be close to this).",
        f"5. **Mapping Quality:** {quality_score}",
        f"6. **Match Details:** {match_details}"
    ]

# Generate and print interpretations
interpretation_bwa = interpret_alignment("alignment_bwa_chrY_random.sam", "BWA-MEM")
interpretation_bowtie2 = interpret_alignment("alignment_bowtie2_chrY_random.sam", "Bowtie2")

print("\n" + "\n".join(interpretation_bwa))
print("\n" + "\n".join(interpretation_bowtie2))


--- **BWA-MEM Alignment Summary (Layman's Terms)** ---
1. **The Question:** Can our 50-base sequence (with 5 unknown parts) be found on Chromosome Y?
2. **The Answer:** The 50-base pair sequence COULD NOT be reliably mapped (found) on Chromosome Y.
3. **Start Location:** N/A
4. **Original Start:** The sequence was taken starting at **22095384** (the map position should be close to this).
5. **Mapping Quality:** N/A
6. **Match Details:** N/A

--- **Bowtie2 Alignment Summary (Layman's Terms)** ---
1. **The Question:** Can our 50-base sequence (with 5 unknown parts) be found on Chromosome Y?
2. **The Answer:** The 50-base pair sequence COULD NOT be reliably mapped (found) on Chromosome Y.
3. **Start Location:** N/A
4. **Original Start:** The sequence was taken starting at **22095384** (the map position should be close to this).
5. **Mapping Quality:** N/A
6. **Match Details:** N/A


In [13]:
# Code Cell 15: Comparison of Mapping Summary Findings (BWA vs. Bowtie2)
def parse_sam_line(sam_file):
    # Reads a SAM file and extracts key fields from the first alignment line
    try:
        with open(sam_file, 'r') as f:
            alignment_line = next((line for line in f if not line.startswith('@')), None)
    except FileNotFoundError:
        return None

    if not alignment_line:
        return None

    fields = alignment_line.split('\t')

    # Extract data, including NM (Edit Distance)
    nm_tag = next((field for field in fields if field.startswith('NM:i:')), 'NM:i:N/A')

    return {
        'pos': fields[3],
        'mapq': int(fields[4]),
        'cigar': fields[5],
        'nm': nm_tag.split(':')[2]
    }

# Parse results for both tools
bwa_data = parse_sam_line("alignment_bwa_chrY_random.sam")
bowtie2_data = parse_sam_line("alignment_bowtie2_chrY_random.sam")

print("--- **Comparison of BWA-MEM and Bowtie2 Alignment Summaries** ---")

if not bwa_data or not bowtie2_data:
    print("Error: Could not parse alignment data for both tools.")
else:
    # 1. Start Position
    pos_match = "Identical" if bwa_data['pos'] == bowtie2_data['pos'] else "Different"
    print(f"1. **Start Position (POS):**")
    print(f"   - BWA-MEM reports position: {bwa_data['pos']}")
    print(f"   - Bowtie2 reports position: {bowtie2_data['pos']}")
    print(f"   -> Finding: Positions are **{pos_match}**.")

    # 2. Mapping Quality (Confidence)
    mapq_diff = bwa_data['mapq'] - bowtie2_data['mapq']
    mapq_finding = f"BWA's score is {abs(mapq_diff)} {'higher' if mapq_diff > 0 else 'lower'}."
    if mapq_diff == 0: mapq_finding = "Scores are identical."

    print(f"\n2. **Mapping Quality (MAPQ):**")
    print(f"   - BWA-MEM confidence score: {bwa_data['mapq']}")
    print(f"   - Bowtie2 confidence score: {bowtie2_data['mapq']}")
    print(f"   -> Finding: {mapq_finding} This reflects differences in how each tool penalizes the 5 unknown 'N' bases.")

    # 3. CIGAR String (Match/Mismatch pattern)
    cigar_match = "Identical" if bwa_data['cigar'] == bowtie2_data['cigar'] else "Different"
    print(f"\n3. **CIGAR String:**")
    print(f"   - BWA-MEM description: {bwa_data['cigar']}")
    print(f"   - Bowtie2 description: {bowtie2_data['cigar']}")
    print(f"   -> Finding: CIGAR strings are **{cigar_match}**. This shows how each tool specifically handled the 5 'N's (e.g., treating them as matches or soft-clips).")

    # 4. Edit Distance (NM Tag)
    print(f"\n4. **Edit Distance (NM):**")
    print(f"   - BWA-MEM value: {bwa_data['nm']}")
    print(f"   - Bowtie2 value: {bowtie2_data['nm']}")
    print(f"   -> Finding: This value (number of differences) reflects the total cost of alignment.")

--- **Comparison of BWA-MEM and Bowtie2 Alignment Summaries** ---
1. **Start Position (POS):**
   - BWA-MEM reports position: 0
   - Bowtie2 reports position: 0
   -> Finding: Positions are **Identical**.

2. **Mapping Quality (MAPQ):**
   - BWA-MEM confidence score: 0
   - Bowtie2 confidence score: 0
   -> Finding: Scores are identical. This reflects differences in how each tool penalizes the 5 unknown 'N' bases.

3. **CIGAR String:**
   - BWA-MEM description: *
   - Bowtie2 description: *
   -> Finding: CIGAR strings are **Identical**. This shows how each tool specifically handled the 5 'N's (e.g., treating them as matches or soft-clips).

4. **Edit Distance (NM):**
   - BWA-MEM value: N/A
   - Bowtie2 value: N/A
   -> Finding: This value (number of differences) reflects the total cost of alignment.


In [14]:
# Code Cell 16: FINAL CHECK: Print the expected start position for easy comparison
with open('random_pos.txt', 'r') as f:
    expected_pos = f.read().strip()
print(f"\n--- Final Verification ---\nExpected Start Position (45 bp sequence): {expected_pos}")
print("Confirm that the 'POS' column in both BWA and Bowtie2 outputs is the same and matches this expected value.")


--- Final Verification ---
Expected Start Position (45 bp sequence): 22095384
Confirm that the 'POS' column in both BWA and Bowtie2 outputs is the same and matches this expected value.


## Detailed Interpretation of BWA vs. Bowtie2 Alignment Findings (Layman's Terms)

Based on the comparison of BWA-MEM and Bowtie2 alignment summaries, here's what we found about how well each tool could locate our 50-base sequence with 5 unknown 'N' parts on Chromosome Y:

1.  **Could the Sequence Be Found?**
    - Both BWA-MEM and Bowtie2 reported that they *could NOT reliably map* or find the 50-base sequence on Chromosome Y.
    - This means neither tool was confident enough to say exactly where our sequence belongs on the chromosome, likely because of the unknown 'N's.

2.  **Reported Start Position (POS):**
    - Both tools reported a start position of `0`.
    - This `0` position usually indicates that the sequence was not successfully mapped to a specific location on the chromosome.
    - Even though the original 45 bp sequence came from a known location (22095384), neither tool confidently found that spot when the 5 'N's were added.

3.  **Confidence Score (MAPQ):**
    - Both tools assigned a confidence score (MAPQ) of `0`.
    - A MAPQ of `0` is the lowest possible score and confirms that the tools have *zero confidence* in the reported mapping location (which was `0`).
    - This low score is a strong indicator that the presence of the 5 'N' bases significantly impacted the tools' ability to find a unique and reliable location for the sequence.

4.  **Alignment Description (CIGAR):**
    - Both tools reported a CIGAR string of `*`.
    - A `*` in the CIGAR field in this context means that there is *no alignment information* available. This is consistent with the sequence not being reliably mapped. If it had mapped, this field would describe how the sequence matched the reference (e.g., how many bases matched, how many were inserted or deleted, or soft-clipped).

5.  **Differences (Edit Distance - NM Tag):**
    - Both tools reported the Edit Distance (NM) as `N/A`.
    - The NM tag tells you the number of differences (substitutions, insertions, or deletions) between the sequence and the spot it mapped to on the chromosome.
    - Since the sequence didn't reliably map, neither tool could calculate an edit distance.

**Overall Conclusion:**

Adding 5 unknown 'N' bases randomly into our 50-base sequence prevented both BWA-MEM and Bowtie2 from confidently and uniquely mapping the sequence back to its original location on Chromosome Y. Both tools indicated a failure to map the sequence reliably, which is reflected in the '0' position, '0' mapping quality, and lack of detailed alignment (CIGAR `*`, NM `N/A`). This highlights how ambiguous bases ('N's) can affect the ability of alignment tools to precisely locate short sequences.

## Detailed Interpretation of BWA vs. Bowtie2 Alignment Findings (Layman's Terms)

Based on the comparison of BWA-MEM and Bowtie2 alignment summaries, here's what we found about how well each tool could locate our 50-base sequence with 5 unknown 'N' parts on Chromosome Y:

1.  **Could the Sequence Be Found?**
    - Both BWA-MEM and Bowtie2 reported that they *could NOT reliably map* or find the 50-base sequence on Chromosome Y.
    - This means neither tool was confident enough to say exactly where our sequence belongs on the chromosome, likely because of the unknown 'N's.

2.  **Reported Start Position (POS):**
    - Both tools reported a start position of `0`.
    - This `0` position usually indicates that the sequence was not successfully mapped to a specific location on the chromosome.
    - Even though the original 45 bp sequence came from a known location (22095384), neither tool confidently found that spot when the 5 'N's were added.

3.  **Confidence Score (MAPQ):**
    - Both tools assigned a confidence score (MAPQ) of `0`.
    - A MAPQ of `0` is the lowest possible score and confirms that the tools have *zero confidence* in the reported mapping location (which was `0`).
    - This low score is a strong indicator that the presence of the 5 'N' bases significantly impacted the tools' ability to find a unique and reliable location for the sequence.

4.  **Alignment Description (CIGAR):**
    - Both tools reported a CIGAR string of `*`.
    - A `*` in the CIGAR field in this context means that there is *no alignment information* available. This is consistent with the sequence not being reliably mapped. If it had mapped, this field would describe how the sequence matched the reference (e.g., how many bases matched, how many were inserted or deleted, or soft-clipped).

5.  **Differences (Edit Distance - NM Tag):**
    - Both tools reported the Edit Distance (NM) as `N/A`.
    - The NM tag tells you the number of differences (substitutions, insertions, or deletions) between the sequence and the spot it mapped to on the chromosome.
    - Since the sequence didn't reliably map, neither tool could calculate an edit distance.

**Overall Conclusion:**

Adding 5 unknown 'N' bases randomly into our 50-base sequence prevented both BWA-MEM and Bowtie2 from confidently and uniquely mapping the sequence back to its original location on Chromosome Y. Both tools indicated a failure to map the sequence reliably, which is reflected in the '0' position, '0' mapping quality, and lack of detailed alignment (CIGAR `*`, NM `N/A`). This highlights how ambiguous bases ('N's) can affect the ability of alignment tools to precisely locate short sequences.